In [1]:
import os

from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
import numpy as np
np.random.seed(1)
import random
random.seed(1)

import pandas as pd
import timeit
import cv2

from skimage.morphology import remove_small_objects

import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from skimage.morphology import square, dilation

In [2]:
# from google.colab import auth
# import sys
# auth.authenticate_user()

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# sys.path.append('/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake')

In [2]:
def list_subdirectories(directory_path):
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return subdirectories

In [4]:
filename = 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_2_part1.png'

def rename_prediction_file(filename,mode):
  # rename to follow the filename guideline in the evaluation file
  city = filename.split('_')[0]
  img_id = '-'.join(filename.split('_disaster_')[1].split('_')[:2])
  if mode == 'localization':
    task = 'localization'
  else:
    task = 'damage'
  newname = '_'.join(['test',task,city+'-'+img_id,'prediction'])+'.png'
  return newname

print(rename_prediction_file(filename,'localization'))
print(rename_prediction_file(filename,'damage'))

test_localization_Adiyaman-0-2_prediction.png
test_damage_Adiyaman-0-2_prediction.png


In [8]:
from tqdm.notebook import tqdm
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

# test_dir = 'test/images'
base_dir = 'turkey_earthquake/'
pred_folders = [base_dir+'predict/res34cls2_1_tuned/']
pred_coefs = [1.0] * 1
loc_folders = [base_dir+'predict/pred34_loc/']
loc_coefs = [1.0] * 1

# sub_folder = base_dir+'predict/submission/'
sub_folder = base_dir+'predict/submission7/'

# _thr = [0.38, 0.13, 0.14] #submission 1
# _thr = [0.19, 0.07, 0.08] #submission 2
# _thr = [0.10, 0.04, 0.05] #submission 3
# _thr = [0.05, 0.02, 0.03] #submission 4
# _thr = [0.01, 0.004, 0.006] #submission 5
# _thr = [0.002, 0.0008, 0.0012] #submission 6
_thr = [0.0002, 0.00008, 0.00012] #submission 7

def process_image(f,city):
    preds = []
    _i = -1
    for pred_folder in pred_folders:
        _i += 1
        d = pred_folder+city+'/'
        msk1 = cv2.imread(path.join(d, f), cv2.IMREAD_UNCHANGED)
        msk2 = cv2.imread(path.join(d, f.replace('_part1', '_part2')), cv2.IMREAD_UNCHANGED)
        msk = np.concatenate([msk1, msk2[..., 1:]], axis=2)
        preds.append(msk * pred_coefs[_i])
    preds = np.asarray(preds).astype('float').sum(axis=0) / np.sum(pred_coefs) / 255

    loc_preds = []
    _i = -1
    for loc_folder in loc_folders:
        _i += 1
        d = loc_folder+city+'/'
        msk = cv2.imread(path.join(d, f), cv2.IMREAD_UNCHANGED)
        loc_preds.append(msk * loc_coefs[_i])
    loc_preds = np.asarray(loc_preds).astype('float').sum(axis=0) / np.sum(loc_coefs) / 255

    loc_preds = loc_preds

    msk_dmg = preds[..., 1:].argmax(axis=2) + 1
    msk_loc = (1 * ((loc_preds > _thr[0]) | ((loc_preds > _thr[1]) & (msk_dmg > 1) & (msk_dmg < 4)) | ((loc_preds > _thr[2]) & (msk_dmg > 1)))).astype('uint8')

    msk_dmg = msk_dmg * msk_loc
    _msk = (msk_dmg == 2)
    if _msk.sum() > 0:
        _msk = dilation(_msk, square(5))
        msk_dmg[_msk & msk_dmg == 1] = 2

    msk_dmg = msk_dmg.astype('uint8')
    # city = f.split('_')[0]

    # old filename convention
    # cv2.imwrite(path.join(sub_folder+city, '{0}.png'.format(f.replace('_pre_disaster_', '_localization_').replace('_part1.png', '_prediction'))), msk_loc, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    # cv2.imwrite(path.join(sub_folder+city, '{0}.png'.format(f.replace('_pre_disaster_', '_damage_').replace('_part1.png', '_prediction'))), msk_dmg, [cv2.IMWRITE_PNG_COMPRESSION, 9])

    #new filename convention
    filename_loc = rename_prediction_file(f,'localization')
    filename_dmg = rename_prediction_file(f,'damage')
    cv2.imwrite(path.join(sub_folder+city, '{0}'.format(filename_loc)), msk_loc, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(path.join(sub_folder+city, '{0}'.format(filename_dmg)), msk_dmg, [cv2.IMWRITE_PNG_COMPRESSION, 9])



if __name__ == '__main__':
    t0 = timeit.default_timer()

    makedirs(sub_folder, exist_ok=True)

    all_files = []
    city_list = list_subdirectories(pred_folders[0])
    for city in city_list:
        print("City:",city)
        # Ensure the directories exist
        pred_folder = pred_folders[0]+city+'/'
        result_folder = sub_folder+city+'/'
        Path(result_folder).mkdir(exist_ok=True)
        for f in tqdm(sorted(listdir(pred_folder))):
          if '_part1.png' in f:
              # all_files.append(f)
              process_image(f,city)
    # with Pool() as pool:
    #   _ = pool.map(process_image, all_files)
    elapsed = timeit.default_timer() - t0
    print('Time: {:.3f} min'.format(elapsed / 60))

City: Adiyaman


  0%|          | 0/322 [00:00<?, ?it/s]

City: Osmaniye


  0%|          | 0/442 [00:00<?, ?it/s]

City: Kahramanmaras


...

City: Antakya


  0%|          | 0/68 [00:00<?, ?it/s]

City: Islahiye


  0%|          | 0/1110 [00:00<?, ?it/s]

Time: 5.198 min


In [ ]:
%%time
import os
# Rename Label files

def rename_files_in_folder(folder_path):
    city_list = list_subdirectories(pred_folders[0])
    for city in city_list:
      print("City:",city)
      # List all files in the folder
      filenames = os.listdir(folder_path+'/'+city)

      # Loop through each file in the folder
      for filename in tqdm(filenames):
          # Generate the new name using the rename_prediction_file function
          newname = rename_prediction_file(filename).replace('.png','',1)

          # Create the full path for the old and new filenames
          old_file_path = os.path.join(folder_path+'/'+city, filename)
          new_file_path = os.path.join(folder_path+'/'+city, newname)

          # Rename the file
          os.rename(old_file_path, new_file_path)
          # print(f"Renamed: {filename} -> {newname}")

# Example usage
folder_path = base_dir+'labels/copernicus_earthquake_turkey_tile_mask_02-23'
rename_files_in_folder(folder_path)

  0%|          | 0/329 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/1110 [00:00<?, ?it/s]

  0%|          | 0/408 [00:00<?, ?it/s]

  0%|          | 0/442 [00:00<?, ?it/s]

  0%|          | 0/1470 [00:00<?, ?it/s]

CPU times: user 737 ms, sys: 448 ms, total: 1.18 s
Wall time: 19.3 s


In [ ]:
%%time
import os
# Rename Label files

def rename_files_in_folder(folder_path):
    city_list = list_subdirectories(pred_folders[0])
    for city in city_list:
      print("City:",city)
      # List all files in the folder
      filenames = os.listdir(folder_path+'/'+city)

      # Loop through each file in the folder
      for filename in tqdm(filenames):
          # Generate the new name using the rename_prediction_file function
          newname = filename.replace('prediction','target',1)

          # Create the full path for the old and new filenames
          old_file_path = os.path.join(folder_path+'/'+city, filename)
          new_file_path = os.path.join(folder_path+'/'+city, newname)

          # Rename the file
          os.rename(old_file_path, new_file_path)
          # print(f"Renamed: {filename} -> {newname}")

# Example usage
folder_path = base_dir+'labels/copernicus_earthquake_turkey_tile_mask_02-23'
rename_files_in_folder(folder_path)

City: Adiyaman


  0%|          | 0/329 [00:00<?, ?it/s]

City: Antakya


  0%|          | 0/68 [00:00<?, ?it/s]

City: Gaziantep


...

City: Osmaniye


  0%|          | 0/442 [00:00<?, ?it/s]

City: Sanliurfa


  0%|          | 0/1470 [00:00<?, ?it/s]

CPU times: user 734 ms, sys: 426 ms, total: 1.16 s
Wall time: 13.3 s


In [ ]:
base_dir = '/content/drive/MyDrive/UCL/DISSERTATION/turkey_earthquake/'
pred_folders = [base_dir+'predict/res34cls2_1_tuned/']
city_list = list_subdirectories(pred_folders[0])
city = city_list[0]
pred_folder = pred_folders[0]+city+'/'
sorted(listdir(pred_folder))[0]

'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_0.tif.png'

In [ ]:
'test_damage_Adiyaman-0-0_prediction.png'.replace('prediction','target',1)

'test_damage_Adiyaman-0-0_target.png'

In [ ]:
path.join(sub_folder, '{0}'.format(f.replace('_pre_', '_localization_').replace('_part1.png', '_prediction')))

'submission/guatemala-volcano_00000021_localization_disaster_part2.png.png'

In [ ]:
all_files

['Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_0_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_10_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_11_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_12_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_13_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_14_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_15_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_17_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_18_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_19_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_1_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_20_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_21_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_22_part1.png',
 'Adiyaman_20230118_10300500D8F90C00_pre_disaster_0_23_part1.png

In [ ]:
city_list[0:1]

['Adiyaman']